<a href="https://colab.research.google.com/github/straylight77/data-sci-learning/blob/master/titanic/titanic3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [2]:
# https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy#Step-1:-Define-the-Problem

data_raw = pd.read_csv('https://raw.githubusercontent.com/straylight77/data-sci-learning/master/titanic/train.csv')
data_val = pd.read_csv('https://raw.githubusercontent.com/straylight77/data-sci-learning/master/titanic/test.csv')

data1 = data_raw.copy(deep=True)
all_data = [data1, data_val]

data_raw.sample(10)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
647,648,1,1,"Simonius-Blumer, Col. Oberst Alfons",male,56.0,0,0,13213,35.5000,A26,C
536,537,0,1,"Butt, Major. Archibald Willingham",male,45.0,0,0,113050,26.5500,B38,S
102,103,0,1,"White, Mr. Richard Frasar",male,21.0,0,1,35281,77.2875,D26,S
814,815,0,3,"Tomlin, Mr. Ernest Portage",male,30.5,0,0,364499,8.0500,NaN,S
399,400,1,2,"Trout, Mrs. William H (Jessie L)",female,28.0,0,0,240929,12.6500,NaN,S
352,353,0,3,"Elias, Mr. Tannous",male,15.0,1,1,2695,7.2292,NaN,C
823,824,1,3,"Moor, Mrs. (Beila)",female,27.0,0,1,392096,12.4750,E121,S
277,278,0,2,"Parkes, Mr. Francis ""Frank""",male,NaN,0,0,239853,0.0000,NaN,S
799,800,0,3,"Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...",female,30.0,1,1,345773,24.1500,NaN,S
781,782,1,1,"Dick, Mrs. Albert Adrian (Vera Gillespie)",female,17.0,1,0,17474,57.0000,B20,S


In [3]:
# COMPLETE: fill in null values 
for ds in all_data:
    ds['Age'].fillna( ds['Age'].median(), inplace=True )
    ds['Embarked'].fillna( ds['Embarked'].mode()[0], inplace=True )
    ds['Fare'].fillna( ds['Fare'].median(), inplace=True )

drop_columns = ['PassengerId', 'Cabin', 'Ticket']
data1.drop(drop_columns, axis=1, inplace=True)

print(data1.isnull().sum())
print("-"*10)
print(data_val.isnull().sum())

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
----------
PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


In [4]:
# CREATE: Feature Engineering 
for ds in all_data:
    ds['Relatives'] = ds['SibSp'] + ds['Parch']

    ds['IsAlone'] = ds['Relatives'].apply(lambda x: 0 if x > 0 else 1) 

    # qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
    # https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
    ds['FareBin'] = pd.qcut(ds['Fare'], 4)
    
    # https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
    ds['AgeBin'] = pd.cut(ds['Age'].astype(int), 5)

data1.sample(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Relatives,IsAlone,FareBin,AgeBin
527,0,1,"Farthing, Mr. John",male,28.0,0,0,221.7792,S,0,1,"(31.0, 512.329]","(16.0, 32.0]"
683,0,3,"Goodwin, Mr. Charles Edward",male,14.0,5,2,46.9000,S,7,0,"(31.0, 512.329]","(-0.08, 16.0]"
889,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.0000,C,0,1,"(14.454, 31.0]","(16.0, 32.0]"
210,0,3,"Ali, Mr. Ahmed",male,24.0,0,0,7.0500,S,0,1,"(-0.001, 7.91]","(16.0, 32.0]"
744,1,3,"Stranden, Mr. Juho",male,31.0,0,0,7.9250,S,0,1,"(7.91, 14.454]","(16.0, 32.0]"
272,1,2,"Mellinger, Mrs. (Elizabeth Anne Maidment)",female,41.0,0,1,19.5000,S,1,0,"(14.454, 31.0]","(32.0, 48.0]"
752,0,3,"Vande Velde, Mr. Johannes Joseph",male,33.0,0,0,9.5000,S,0,1,"(7.91, 14.454]","(32.0, 48.0]"
264,0,3,"Henry, Miss. Delia",female,28.0,0,0,7.7500,Q,0,1,"(-0.001, 7.91]","(16.0, 32.0]"
706,1,2,"Kelly, Mrs. Florence ""Fannie""",female,45.0,0,0,13.5000,S,0,1,"(7.91, 14.454]","(32.0, 48.0]"
764,0,3,"Eklund, Mr. Hans Linus",male,16.0,0,0,7.7750,S,0,1,"(-0.001, 7.91]","(-0.08, 16.0]"


In [5]:
# CONVERT: change to category types 
from sklearn import preprocessing

label = preprocessing.LabelEncoder()
for ds in all_data:    
    ds['Sex_Code'] = label.fit_transform(ds['Sex'])
    ds['Embarked_Code'] = label.fit_transform(ds['Embarked'])
    #ds['Title_Code'] = label.fit_transform(ds['Title'])
    ds['AgeBin_Code'] = label.fit_transform(ds['AgeBin'])
    ds['FareBin_Code'] = label.fit_transform(ds['FareBin'])

data1.sample(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Relatives,IsAlone,FareBin,AgeBin,Sex_Code,Embarked_Code,AgeBin_Code,FareBin_Code
753,0,3,"Jonkoff, Mr. Lalio",male,23.0,0,0,7.8958,S,0,1,"(-0.001, 7.91]","(16.0, 32.0]",1,2,1,0
839,1,1,"Marechal, Mr. Pierre",male,28.0,0,0,29.7000,C,0,1,"(14.454, 31.0]","(16.0, 32.0]",1,0,1,2
324,0,3,"Sage, Mr. George John Jr",male,28.0,8,2,69.5500,S,10,0,"(31.0, 512.329]","(16.0, 32.0]",1,2,1,3
22,1,3,"McGowan, Miss. Anna ""Annie""",female,15.0,0,0,8.0292,Q,0,1,"(7.91, 14.454]","(-0.08, 16.0]",0,1,0,1
443,1,2,"Reynaldo, Ms. Encarnacion",female,28.0,0,0,13.0000,S,0,1,"(7.91, 14.454]","(16.0, 32.0]",0,2,1,1
62,0,1,"Harris, Mr. Henry Birkhardt",male,45.0,1,0,83.4750,S,1,0,"(31.0, 512.329]","(32.0, 48.0]",1,2,2,3
846,0,3,"Sage, Mr. Douglas Bullen",male,28.0,8,2,69.5500,S,10,0,"(31.0, 512.329]","(16.0, 32.0]",1,2,1,3
402,0,3,"Jussila, Miss. Mari Aina",female,21.0,1,0,9.8250,S,1,0,"(7.91, 14.454]","(16.0, 32.0]",0,2,1,1
454,0,3,"Peduzzi, Mr. Joseph",male,28.0,0,0,8.0500,S,0,1,"(7.91, 14.454]","(16.0, 32.0]",1,2,1,1
178,0,2,"Hale, Mr. Reginald",male,30.0,0,0,13.0000,S,0,1,"(7.91, 14.454]","(16.0, 32.0]",1,2,1,1
